# ДЗ 1

## Задание 1

Сравните метрики hit_rate@k, precision@k. Какую матрику использовать предпочтительно и почему. Привидите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?

### Ответ

hit_rate@k - это наличие хотя бы одного релевантного товара из k рекомендованных. <br>precision@k - это доля релевантных товаров из k рекомендованных. 
- Если у нас потоковые рекомендации, например треки в плейлисте или товары массового потребления на Ozon, то нас интересует доля, т.е. precision@k
- Если мы рекомендуме товар штучного спроса, квартиры, машины, курсы на GB :), то нас интересует сам факт интереса потенциального клиента к данной категории товаров, т.е. hit_rate@k

## Задание 2

В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в знаменателе присутствует логарифм.

### Ответ

Логарифм даёт "сглаживание". В контексте NDCG@k даёт меньший штраф при ошибке ранжирования. Наряду с логарифмом можно использовать сигмоид, возможно гиперболический тангенс. Логарифм в функции потерь даёт удобство дифференцирования. 

## Задание 3

Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики и чем являются интеракции, чтобы она могла быть использована).

### Ответ

#### Непрерывные метрики:
Оценка подобия клиентов или таргетов (товаров, треков в плейлисте и т.д.). При пересчёте взаимодействия в валовые единицы (денежные или объёмные единицы измерения). 
- MAE (Mean Absolute Error) Среднее абсолютное отклонение
- MSE (Mean Squared Error) Среднеквадратичная ошибка
- RMSE (Root Mean Squared Error) Корень из среднеквадратичной ошибки

#### Дискретные метрики:
Используются при учёте дискретных взаимодействий типа: клиент-товар, товар-товар, товар-клиент (обращение клиета к карточкам и разделам каталога, приобретение товара):
- Precision Доля рекомендаций, понравившихся пользователю
- Recall Доля интересных пользователю товаров, которая показана
- F1-Measure Среднее гармоническое метрик Precision и Recall. Полезно, когда заранее невозможно сказать, какая из метрик важнее
- ROC AUC Насколько высока концентрация интересных товаров в начале списка рекомендаций
- Precision@N Метрика Precision, посчитанная на Top-N записях
- Recall@N Метрика Recall, посчитанная на Top-N записях
- AverageP Среднее значение Precision на всем списке рекомендаций

#### Метрики ранжирования (Rank Accuracy):
Необходимы при оценки релевантности в зависимости от позиции
- Mean Reciprocal Rank На какой позиции списка рекомендаций пользователь находит первую полезную
- Spearman Correlation Корреляция (Спирмена) реального и прогнозируемого рангов рекомендаций
- nDCG Информативность выдачи с учетом ранжирования рекомендаций
- Fraction of Concordance Pairs Насколько высока концентрация интересных товаров в начале списка рекомендаций

## Задание 4

In [10]:
boughted = [1, 3, 5, 7, 9, 11]
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]
k = 8

Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, RR@8, ERR@8

In [11]:
import numpy as np

### pr@8

In [12]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    print(flags)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [13]:
precision_at_k(recommended, boughted, k)

[False False  True  True  True  True]


0.5

### rec@8

In [14]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [15]:
recall_at_k(recommended, boughted, k)

0.6666666666666666

### AP@8

In [26]:
def ap_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1): 
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

In [27]:
ap_at_k(recommended, boughted, k)

[False False False False False False]
[False False  True False False False]
[False False  True  True False False]
[False False  True  True False  True]


0.32

### NDCG@8

### RR@8

In [28]:
def reciprocal_rank_at_k(recommended_list, bought_list, k=8):
    ranks=0.
    for item_rec in recommended_list[:k]:
        for i, item_bought in enumerate(bought_list):
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    return ranks / len(recommended_list)

In [29]:
reciprocal_rank_at_k(recommended, boughted, k)

0.095

### ERR@8